### Exercise 1:
Using the file above

- Open the file and read the WCS
- Display the bounding_box
- Generate the grid of inputs Extra bonus
  - Generate a grid using the edge of the pixels instead of the centers (hint: use the center parameter)
  - Generate a grid with sampling every 5 pixels (hint: use the step parameter)
- Use the Shared and Legacy APIs to generate world coordinates
- Evaluate the WCS on x=4088 and y=4088
- Compute the footprint of the image on sky

In [ ]:
from roman_datamodels import datamodels as rdm
cal = rdm.open('../asdf/data/roman.asdf')

In [ ]:
w = cal.meta.wcs
print(f"Bounding box : \n\n {w.bounding_box}")

In [ ]:
from gwcs.wcstools import grid_from_bounding_box

x, y = grid_from_bounding_box(w.bounding_box)
print(x)
print(y)

In [ ]:
x_edge, y_edge = grid_from_bounding_box(w.bounding_box, center=False)
print(x_edge)
print('\n\n')
print(y_edge)

In [ ]:
x_edge_5, y_edge_5 = grid_from_bounding_box(w.bounding_box, center=False, step=5)
print(x_edge_5)
print('\n\n')
print(y_edge_5)

In [ ]:
# Use the Shared and Legacy APIs to generate world coordinates

ra, dec = w(x_edge_5, y_edge_5)
print(ra, dec)

In [ ]:
sky = w.pixel_to_world(x, y)
print(sky)

In [ ]:
# Evaluate the WCS on x=4088 and y=4088 (note these are outside the bounding box)

w(4088, 4088)

In [ ]:
# Compute the footprint of the image on sky

print(f"Footprint on sky: \n {w.footprint()}")

### Exercise 2:

The goal of this exercise is to introduce astropy.modeling which is at the core of the GWCS implementation.
- Read the jwst file and print the forward transform.
- Use the prescription above to generate a compound model with different parameters.
- Using this new transform generate a GWCS object. The coordinate frames can be accessed by `wcs.input_frame` and `wcs.output_frame`.

In [ ]:
from astropy.modeling import models as astm

from stdatamodels.jwst import datamodels

In [ ]:
jcal = datamodels.open('../asdf/data/jwst.fits')

In [ ]:
jw = jcal.meta.wcs
print(jw.forward_transform)

In [ ]:
# astm.

`astropy.modeling` has many predefined models and a machinery to combine them in difference ways. There are some WCS specific models within `gwcs`.

The "join" operator, `&`, evaluates the models on independent inputs and concatenates the result. In the example below the name of the parameter, `offset` is omitted. Models have an attribute `param_names` which shows the orider and names of parameters.

In [ ]:
print(astm.AffineTransformation2D.param_names)

print(astm.Shift.param_names)

In [ ]:
shifts = astm.Shift(offset=-10) & astm.Shift(-10.4)
print (shifts(1, 2))

In [ ]:
affine = astm.AffineTransformation2D(matrix=[[0.31671001, -0.94852241], [-0.94852241, -0.31671001]], translation=[0., 0.])

scales = astm.Scale(.00011) & astm.Scale(.00012)

tan = astm.Pix2Sky_TAN()

n2c = astm.RotateNative2Celestial(lon=56.3, lat=-72.1, lon_pole=180)

The "composition" operator, `|`, combines models in series - the output of one model is the input to the next one.

The combined model below represents a typical WCS imaging transform using a tangent projection.

In [ ]:
forward = shifts | affine | scales | tan | n2c

print(forward)

In [ ]:
from gwcs import wcs
from gwcs import coordinate_frames as cf

from astropy import units as u
from astropy.coordinates import ICRS

In [ ]:
# Let's generate an input and output frames:

detector = cf.Frame2D(name='detector', axes_names=('x', 'y'), unit=(u.ppix, u.pix))

celestial = cf.CelestialFrame(name='icrs', reference_frame=ICRS(), axes_names=('ra', 'dec'), unit=(u.deg, u.deg))

In [ ]:
wnew = wcs.WCS(input_frame=detector, output_frame=celestial, forward_transform=forward)

In [ ]:
wnew.invert(56.3, -72.1)

##### More on combining models

- Models can be combined using arithmetic operators, `+`, `-`, `*`, `/`, `**`. In this case all models are evaluated on the same inputs and the results are combined using the operator.
- There are two special models
  - `astm.Identity` passes the inputs unchanged
  - `astm.Mapping` takes a tuple which is an index on the inputs and rearranges them, or drops an input.

In [ ]:
im = astm.Identity(2)

print(im(10, 13))

In [ ]:
mm = astm.Mapping((0, 0, 2), n_inputs=4)
mm(0, 1, 2, 3)